In [1]:
from google.colab import files
uploaded = files.upload()

Saving test_dense_noprocess.csv to test_dense_noprocess.csv
Saving train_dence_noprocess.csv to train_dence_noprocess.csv


In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingGridSearchCV
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import cv

In [3]:
dense_median_nona = pd.read_csv('train_dence_noprocess.csv')
print(dense_median_nona.shape)
dense_median_nona #feature output

(7715, 132)


,Id,V1_1,V2_1,V3_1,V4_1,V5_1,V6_1,V7_1,V8_1,V9_1,V10_1,V11_1,V12_1,V13_1,V14_1,V15_1,V16_1,V17_1,V18_1,V19_1,V20_1,V21_1,V22_1,P1_1,P1_2,P1_3,P1_4,P2_1,P2_2,P2_3,P2_4,P3_1,P3_2,P3_3,P3_4,P4_1,P4_2,P4_3,P4_4,P5_1,...,P18_2,P18_3,P18_4,P19_1,P19_2,P19_3,P19_4,P20_1,P20_2,P20_3,P20_4,P21_1,P21_2,P21_3,P21_4,P22_1,P22_2,P22_3,P22_4,P23_1,P23_2,P23_3,P23_4,P24_1,P24_2,P24_3,P24_4,P25_1,P25_2,P25_3,P25_4,P26_1,P26_2,P26_3,P26_4,P27_1,P27_2,P27_3,P27_4,target
0,-6536978109522202983,0.000000,0.0,0.000000,0.011364,0.006579,0.0311,0.013158,0.006579,0.228694,0.000000,0.333073,0.000000,0.0,0.019737,0.064203,0.263279,0.011364,0.000000,0.000000,0.000000,0.010870,0.000000,86.0,64.0,59.0,67.0,24137.0,43316.0,19961.0,40729.0,0.666667,0.347826,0.500000,0.473684,5.205102,3.926019,3.627770,3.348995,77.950000,...,14078.000000,10814.454545,13842.052632,15.400000,17.016667,41.100000,33.850000,4.82,4.82,4.82,4.82,8.911111,8.292754,9.753030,10.520614,200.0,510.0,190.0,382.0,0.000000,0.000000,0.000000,0.026316,4.820000,4.8200,4.820000,4.814737,0.333333,0.652174,0.500000,0.447368,6712.0,4873.0,2175.0,4093.0,0.333333,0.391304,0.181818,0.394737,0.0
1,-1744017237843019509,0.051282,1.0,0.014363,0.018308,0.037632,0.0000,0.180246,0.103449,0.379082,0.000000,0.020292,0.000000,0.0,0.000000,0.000000,0.055489,0.071362,0.071817,0.000000,0.047960,0.000000,0.000000,47.0,53.0,52.0,95.0,56986.0,55645.0,31499.0,41773.0,0.304348,0.228571,0.181818,0.250000,3.916232,4.911539,2.900853,4.283126,30.466667,...,14424.361111,14787.181818,15868.678571,31.866667,44.600000,30.266667,21.783333,4.94,4.91,4.92,4.93,9.438406,9.803810,10.302273,9.141071,777.0,935.0,591.0,541.0,0.021739,0.085714,0.045455,0.035714,4.908478,4.9100,4.915909,4.905357,0.673913,0.685714,0.772727,0.714286,1296.0,2111.0,2620.0,5640.0,0.239130,0.388889,0.636364,0.607143,0.0
2,-9187108666132882725,0.307692,9.0,0.000000,0.003521,0.010830,0.0000,0.526146,0.241712,0.096512,0.000000,0.007042,0.007042,0.0,0.003521,0.005000,0.009335,0.005000,0.010814,0.000000,0.073524,0.000000,0.000000,45.0,45.0,55.0,52.0,65124.0,41206.0,38611.0,65501.0,0.153846,0.162791,0.385714,0.160000,2.980966,2.505911,2.424077,2.648700,23.633333,...,6432.825581,8591.323944,9337.640000,16.100000,23.250000,14.416667,14.016667,4.99,4.99,4.99,4.99,6.357949,7.240891,6.536190,6.601000,534.0,371.0,518.0,529.0,0.015385,0.000000,0.028571,0.000000,4.990000,4.9900,4.990000,4.990000,0.830769,0.837209,0.585714,0.840000,1235.0,978.0,1322.0,2034.0,0.181818,0.139535,0.225352,0.180000,0.0
3,-1259418257712246678,0.105263,2.0,0.005682,0.062350,0.000000,0.0000,0.252071,0.276184,0.056668,0.000000,0.042382,0.000000,0.0,0.005682,0.009259,0.000000,0.163985,0.020536,0.000000,0.105201,0.000000,0.000000,60.0,70.0,74.0,97.0,23956.0,37259.0,32117.0,22008.0,1.000000,1.000000,1.000000,1.000000,2.959961,2.770644,2.490932,2.849632,29.833333,...,10688.659091,11074.550000,11358.703704,56.266667,32.016667,37.233333,22.366667,5.00,5.00,5.00,5.00,12.476190,9.537121,9.987500,8.929012,399.0,609.0,530.0,463.0,0.000000,0.000000,0.000000,0.000000,5.000000,5.0000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,2286.0,2040.0,3155.0,3929.0,0.571429,0.590909,0.625000,0.703704,0.0
4,7132677487852039192,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.082369,0.000000,0.202965,0.000000,0.0,0.000000,0.000000,0.714666,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.0,51.0,51.0,51.0,12677.0,9113.0,10431.0,9252.0,0.218182,0.137255,0.173913,0.204545,3.211575,3.134223,3.404928,3.608876,18.683333,...,4116.058824,4049.956522,4380.295455,33.150000,17.933333,12.700000,14.616667,4.98,4.98,4.98,4.95,7.021818,8.034314,7.252899,6.840909,164.0,135.0,122.0,135.0,0.000000,0.000000,0.000000,0.000000,4.978571,4.9800,4.968261,4.950000,0.781818,0.862745,0.826087,0.795455,1058.0,1365.0,1662.0,912.0,0.017857,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [4]:
test_dense_median_nona = pd.read_csv('test_dense_noprocess.csv')
print(test_dense_median_nona.shape)
test_dense_median_nona #feature output

(3307, 131)


,Id,V1_1,V2_1,V3_1,V4_1,V5_1,V6_1,V7_1,V8_1,V9_1,V10_1,V11_1,V12_1,V13_1,V14_1,V15_1,V16_1,V17_1,V18_1,V19_1,V20_1,V21_1,V22_1,P1_1,P1_2,P1_3,P1_4,P2_1,P2_2,P2_3,P2_4,P3_1,P3_2,P3_3,P3_4,P4_1,P4_2,P4_3,P4_4,P5_1,...,P18_1,P18_2,P18_3,P18_4,P19_1,P19_2,P19_3,P19_4,P20_1,P20_2,P20_3,P20_4,P21_1,P21_2,P21_3,P21_4,P22_1,P22_2,P22_3,P22_4,P23_1,P23_2,P23_3,P23_4,P24_1,P24_2,P24_3,P24_4,P25_1,P25_2,P25_3,P25_4,P26_1,P26_2,P26_3,P26_4,P27_1,P27_2,P27_3,P27_4
0,6107424960806386948,0.000000,0.0,0.014881,0.026786,0.00000,0.011905,0.029762,0.041295,0.354911,0.021577,0.028646,0.000000,0.0,0.028646,0.061384,0.019717,0.171875,0.115327,0.007812,0.041667,0.017857,0.005952,69.0,73.0,50.0,56.0,64212.0,47622.0,48420.0,45352.0,0.097561,0.095238,0.100000,0.031250,4.730041,4.273641,4.067553,5.756356,63.800000,...,21521.666667,22888.809524,21491.678571,20878.656250,24.800000,22.900000,46.050000,23.683333,4.86,4.85,4.85,4.85,10.808537,11.073016,10.405000,9.622396,550.0,719.0,526.0,597.0,0.0,0.000000,0.00000,0.031250,4.85381,4.841429,4.850000,4.850000,0.902439,0.904762,0.900000,0.937500,4321.0,6265.0,2003.0,1069.0,0.642857,0.714286,0.821429,0.656250
1,-1808613790090531322,0.000000,0.0,0.000000,0.193344,0.00000,0.000000,0.305022,0.000000,0.112810,0.000000,0.304820,0.000000,0.0,0.000000,0.072099,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,64.0,56.0,54.0,60.0,12421.0,14491.0,15586.0,14911.0,0.809524,0.818182,0.868421,0.823529,2.209511,2.705761,2.242723,2.891978,22.750000,...,6647.904762,6059.303030,6614.473684,7934.676471,18.300000,18.733333,14.500000,19.300000,4.98,4.98,4.99,4.99,7.250000,8.639899,7.176316,6.993627,193.0,179.0,196.0,194.0,0.0,0.000000,0.00000,0.000000,4.98000,4.980000,4.989474,4.990000,0.190476,0.181818,0.131579,0.176471,2416.0,1538.0,2198.0,2811.0,0.095238,0.030303,0.131579,0.147059
2,-2649749947427248381,0.000000,0.0,0.000000,0.016667,0.00000,0.000000,0.199603,0.229365,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.027778,0.000000,0.000000,0.215873,0.000000,0.277381,0.033333,0.000000,49.0,100.0,63.0,65.0,14659.0,4691.0,11357.0,14969.0,0.444444,1.000000,0.142857,0.200000,2.750147,1.585480,3.413373,4.139686,36.483333,...,8088.722222,3795.666667,8926.571429,7083.733333,17.583333,10.866667,12.366667,9.950000,4.98,4.98,4.98,4.98,8.680556,8.455556,9.514286,6.848889,144.0,115.0,198.0,161.0,0.0,0.000000,0.00000,0.000000,4.98000,4.980000,4.980000,4.980000,0.000000,0.000000,0.000000,0.000000,2639.0,2057.0,1985.0,3213.0,0.000000,0.000000,0.285714,0.066667
3,4684257435812939224,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.135870,0.085870,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.243478,0.000000,0.000000,0.000000,0.000000,0.534783,0.000000,0.000000,70.0,110.0,91.0,90.0,19194.0,18954.0,20964.0,21904.0,0.000000,0.200000,0.000000,0.000000,3.306168,3.612487,3.089151,3.987020,23.116667,...,6554.739130,10296.000000,10993.200000,13567.250000,22.816667,11.133333,18.916667,11.650000,4.97,4.97,4.97,4.97,8.920290,8.510000,8.053030,7.879167,233.0,198.0,189.0,250.0,0.0,0.000000,0.00000,0.000000,4.97000,4.970000,4.970000,4.970000,1.000000,0.800000,1.000000,1.000000,1266.0,6856.0,6248.0,4464.0,0.130435,0.200000,0.200000,0.375000
4,-7030158396775705152,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.036871,0.031552,0.134277,0.000000,0.092087,0.000000,0.0,0.163709,0.498676,0.000000,0.000000,0.000000,0.000000,0.037509,0.005319,0.000000,48.0,69.0,60.0,62.0,29043.0,25137.0,23388.0,19672.0,0.224138,0.458333,0.240000,0.344828,2.074779,1.799341,2.205403,2.017794,41.666667,...,6723.517241,9581.212766,8900.480000,8179.931034,69.700000,197.066667,22.150000,68.650000,4.88,4.88,4.90,4.90,15.784770,21.888194,11.696667,15.417816,291.0,341.0,241.0,250.0,0.0,0.000000,0.00000,0.000000,4.88000,4.880000,4.893600,4.900000,0.775862,0.541667,0.760000,0.655172,1613.0,1669.0,2897.0,3003.0,0.103448,0.212766,0.120000,0.103448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [42]:
X_train = dense_median_nona.drop(['target', 'Id'], axis = 1)
Y_train = dense_median_nona['target']

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)

In [43]:
X_test = test_dense_median_nona.drop(['Id'], axis = 1)

In [139]:
xgtrain = xgb.DMatrix(X_train, label=Y_train)
model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', learning_rate=2, max_delta_step=0, max_depth=11, scale_pos_weight=0.209437216,
              min_child_weight=0, num_parallel_tree=5, n_estimators=100, seed=13, gamma=0, eta=0.3, sampling_method='gradient_based', objective='reg:logistic',
              subsample=0.7777777777788, # is the fraction of the training samples (randomly selected) that will be used to train each tree
              colsample_bytree=0.7777777777788, # is the fraction of features (randomly selected) that will be used to train each tree
              colsample_bylevel=0.3 # is the fraction of features (randomly selected) that will be used in each node to train each tree
              )
xgb_param = model.get_xgb_params()
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=1500, nfold=3, metrics=['auc'],
     early_stopping_rounds=50, stratified=True, seed=13)
print('Best number of trees = {}'.format(cvresult.shape[0]))
model.set_params(n_estimators=cvresult.shape[0])
print('Fit on the trainings data')
model.fit(X_train, Y_train, eval_metric='auc')
print('Overall AUC:', roc_auc_score(Y_train, model.predict_proba(X_train)[:,1]))

Start cross validation
Best number of trees = 1500
Fit on the trainings data
Overall AUC: 1.0


In [137]:
Y_pred = model.predict(X_train, ntree_limit=cvresult.shape[0])
Y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [138]:
accuracy = accuracy_score(Y_train, Y_pred)
print('Accuracy on training data: ', accuracy)

Y_pred_val = model.predict(X_val)
accuracy = accuracy_score(Y_val, Y_pred_val)
print('Accuracy on validation data: ', accuracy)

pred_test = model.predict_proba(X_test, ntree_limit=cvresult.shape[0])

Accuracy on training data:  1.0
Accuracy on validation data:  0.9334773218142549


In [111]:
pred_test = pd.DataFrame(data = pred_test)

In [112]:
pred_test[0].shape[0]

3307

In [ ]:
id = test_dense_median_nona['Id']
result = pd.concat([id, pred_test], axis=1)
result = result.drop(result.columns[1], axis=1)
result = result.rename(columns={1 : 'Predicted'})
result

In [114]:
result.to_csv('result.csv', index=False) 
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>